In [52]:
import yfinance as yf
import pandas as pd
import numpy as np
import networkx as nx
import plotly.graph_objs as go
import plotly.express as px
import streamlit as st

sp500_tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
tickers = sp500_tickers['Symbol'].to_list()

data = yf.download(tickers, start='2020-01-01', end='2024-06-01')['Adj Close']

[*********************100%%**********************]  503 of 503 completed

2 Failed downloads:
['BF.B']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2020-01-01 -> 2024-06-01)')
['BRK.B']: Exception('%ticker%: No timezone found, symbol may be delisted')


In [53]:
returns = data.pct_change(fill_method = None).iloc[1:].dropna(axis = 1)

In [85]:
#returns.to_parquet('../Data/SP500_Returns.parquet')

In [71]:
aapl_corr = returns.corrwith(returns['AAPL'])

threshold = 0.6
related_stocks = aapl_corr[aapl_corr > threshold].index.to_list()

In [72]:
related_returns = returns[related_stocks]
corr_matrix = related_returns.corr()

distance_matrix = 1 - corr_matrix

G = nx.Graph()
for i in range(len(distance_matrix.columns)):
    for j in range(i+1, len(distance_matrix.columns)):
        G.add_edge(distance_matrix.columns[i], distance_matrix.columns[j], weight=distance_matrix.iloc[i, j])

In [73]:
mst = nx.minimum_spanning_tree(G)
pos = nx.spring_layout(mst)
pos['AAPL'] = np.array([0, 0])

In [81]:
ticker = 'AAPL'
edge_x = []
edge_y = []
for edge in mst.edges(data=True):
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=1, color='#888'),
    hoverinfo='none',
    mode='lines'
)

# 노드 추출
node_x = []
node_y = []
node_text = []
for node in mst.nodes():
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)
    node_text.append(node)

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers+text',
    text=node_text,
    textposition="bottom center",
    hoverinfo='text',
    marker=dict(
        showscale=True,
        colorscale='YlGnBu',
        size=10,
        color=[],
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        )
    )
)

fig = go.Figure(data=[edge_trace, node_trace],
                layout=go.Layout(
                    title=f'{ticker} related Securities',
                    titlefont_size=16,
                    showlegend=False,
                    hovermode='closest',
                    width=600,
                    height=400,
                    margin=dict(b=20, l=5, r=5, t=40),
                    annotations=[dict(
                        text=f"Network between {ticker} and S&P 500",
                        showarrow=False,
                        xref="paper", yref="paper",
                        x=0.005, y=-0.002
                    )],
                    xaxis=dict(showgrid=False, zeroline=False),
                    yaxis=dict(showgrid=False, zeroline=False))
                )

In [82]:
fig.show()